In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv('./data/PCA_Output.csv').drop('Unnamed: 0', axis=1)
df.head()

,PC-1,PC-2,PC-3,PC-4,PC-5,PC-6,PC-7,PC-8,PC-9,PC-10,...,PC-14,PC-15,PC-16,PC-17,PC-18,PC-19,PC-20,Year,Round,School
0,6.076648,-1.976649,-2.848498,1.619673,-1.499366,-0.361836,1.019153,1.434457,0.714081,0.811379,...,0.688366,-0.304890,-0.281731,-0.205803,-0.140139,0.340214,-0.099955,0,6,Appalachian State
1,-5.397574,-1.225701,-2.640536,-0.686919,-2.231943,1.268349,-0.970858,0.854973,0.483640,0.945111,...,0.661443,0.159350,0.994632,0.590591,0.674845,0.279921,-0.757373,0,5,Arizona
2,-0.459077,-0.226509,-0.177002,-0.025225,-2.348304,-2.840060,-1.581769,-1.759809,0.738214,0.328368,...,-0.694507,-0.067370,-0.248602,0.238970,-0.456775,0.480758,-0.203236,0,6,Arkansas
3,0.737941,-2.735202,-1.689384,-1.009387,-1.348711,-2.272617,-0.260527,0.004799,-0.184886,1.096526,...,0.271134,0.544804,-0.797072,-0.432104,-0.827898,0.256052,-0.232796,0,5,Auburn
4,3.519914,2.074051,-0.954414,-0.144123,-1.547389,-1.325878,1.202000,2.277089,-1.031561,-0.126343,...,-1.376886,-1.040308,0.260123,-0.111515,-0.763786,0.158143,0.004584,0,6,Ball State


In [5]:
df_test = df.sample(frac=.2, replace=False, random_state=0).copy()
df_train = df.drop(df_test.index).copy()

features_train = df_train.drop(['Year', 'Round', 'School'], axis=1)
features_test = df_test.drop(['Year', 'Round', 'School'], axis=1)

features_train = (features_train - features_train.mean())/features_train.std()
features_test = (features_test - features_test.mean())/features_test.std()

targets_train = df_train.Round
targets_test = df_test.Round

In [6]:
svc = SVC(C=.0001, kernel='rbf')
rfc = RandomForestClassifier(max_depth=17, n_estimators=1900)
lgr = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=53)
dtr = DecisionTreeClassifier(max_depth=1)
gbt = GradientBoostingClassifier(learning_rate=.00001, max_depth=2, n_estimators=75)

In [14]:
voting = VotingClassifier(estimators=[('svc', svc), ('lgr', lgr), ('knn', knn), ('dtr', dtr), ('gbt', gbt)])

In [15]:
voting.fit(features_train, targets_train)

VotingClassifier(estimators=[('svc', SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('lgr', LogisticRegression(C=1.0, class_weight=Non...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [17]:
voting.score(features_train, targets_train)

0.4676165803108808

In [18]:
voting.score(features_test, targets_test)

0.5129533678756477